<a href="https://colab.research.google.com/github/deepd1534/data-science-portfolio/blob/master/Covid_19_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing all necessary libraries


In [1]:
# IMPORTING ALL NECESSARY LIBRARIES
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import requests
from datetime import datetime
import json
import plotly.express as px
import plotly.io as pio
import plotly

Sending a request to the website for COVID-19 data

In [2]:
#READING DATA FROM WEBSITE
raw= requests.get("https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1/query?where=1%3D1&outFields=*&outSR=4326&f=json")
raw_json = raw.json()
df = pd.DataFrame(raw_json["features"])

In [3]:
df.head()

,attributes,geometry
0,"{'OBJECTID': 1, 'Province_State': None, 'Count...","{'x': 67.70995300000004, 'y': 33.93911000000003}"
1,"{'OBJECTID': 2, 'Province_State': None, 'Count...","{'x': 20.168300000000045, 'y': 41.15330000000006}"
2,"{'OBJECTID': 3, 'Province_State': None, 'Count...","{'x': 1.6596000000000686, 'y': 28.033900000000..."
3,"{'OBJECTID': 4, 'Province_State': None, 'Count...","{'x': 1.5218000000000416, 'y': 42.50630000000007}"
4,"{'OBJECTID': 5, 'Province_State': None, 'Count...","{'x': 17.87390000000005, 'y': -11.202699999999..."


Transforming and Cleaning Data

In [4]:
data_list = df["attributes"].tolist() 
df_final = pd.DataFrame(data_list)
df_final.set_index("OBJECTID")
df_final = df_final[["Country_Region", "Province_State", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Last_Update"]]
df_final

,Country_Region,Province_State,Lat,Long_,Confirmed,Deaths,Recovered,Last_Update
0,Afghanistan,None,33.939110,67.709953,69130,2881,57119,1.622273e+12
1,Albania,None,41.153300,20.168300,132285,2448,129097,1.622273e+12
2,Algeria,None,28.033900,1.659600,128198,3455,89232,1.622273e+12
3,Andorra,None,42.506300,1.521800,13693,127,13416,1.622273e+12
4,Angola,None,-11.202700,17.873900,33944,749,27577,1.622273e+12
...,...,...,...,...,...,...,...,...
762,US,Ohio,40.388783,-82.764915,1100972,19861,0,1.622273e+12
763,US,Alabama,32.318200,-86.902300,543405,11146,0,1.622273e+12
764,US,North Carolina,35.630066,-79.806419,1001154,13078,0,1.622273e+12
765,US,District of Columbia,38.897438,-77.026817,48898,1132,0,1.622273e+12


Selecting India

In [5]:
df_india = df_final[df_final['Country_Region'] == 'India']

In [6]:
df_india.drop(columns = 'Country_Region',inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [7]:
df_india.rename(columns = {'Province_State' : 'State', 'Long_': 'Long'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
df_india = df_india.reset_index(drop = True)
df_india.head(5)

,State,Lat,Long,Confirmed,Deaths,Recovered,Last_Update
0,Andaman and Nicobar Islands,11.225999,92.968178,6936,113,6630,1.622273e+12
1,Andhra Pradesh,15.912900,79.740000,1657986,10634,1466990,1.622273e+12
2,Arunachal Pradesh,27.768456,96.384277,26317,111,22352,1.622273e+12
3,Assam,26.357149,92.830441,398010,3168,340178,1.622273e+12
4,Bihar,25.679658,85.604840,702682,5004,672868,1.622273e+12


In [9]:

def convertTime(t): #A function convertTime is written to convert the timestamp into a date with format.
    t = int(t)
    return datetime.fromtimestamp(t)



df_india = df_india.dropna(subset=["Last_Update"])
df_india["State"].fillna(value="", inplace=True)

df_india["Last_Update"]= df_india["Last_Update"]/1000
df_india["Last_Update"] = df_india["Last_Update"].apply(convertTime)


In [10]:

df_india['State'][0] = 'Andaman and Nicobar'
df_india['State'][7] = 'Dadra and Nagar Haveli'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [36]:
df_india.drop([17,33],axis=0,inplace = True)


In [37]:
df_india

,State,Lat,Long,Confirmed,Deaths,Recovered,Last_Update,text
0,Andaman and Nicobar,11.225999,92.968178,6936,113,6630,2021-05-29 07:20:52,Andaman and Nicobar<br>Confirmed: 6936<br>Deat...
1,Andhra Pradesh,15.912900,79.740000,1657986,10634,1466990,2021-05-29 07:20:52,Andhra Pradesh<br>Confirmed: 1657986<br>Deaths...
2,Arunachal Pradesh,27.768456,96.384277,26317,111,22352,2021-05-29 07:20:52,Arunachal Pradesh<br>Confirmed: 26317<br>Death...
3,Assam,26.357149,92.830441,398010,3168,340178,2021-05-29 07:20:52,Assam<br>Confirmed: 398010<br>Deaths: 3168<br>...
4,Bihar,25.679658,85.604840,702682,5004,672868,2021-05-29 07:20:52,Bihar<br>Confirmed: 702682<br>Deaths: 5004<br>...
5,Chandigarh,30.733839,76.768278,59577,735,56084,2021-05-29 07:20:52,Chandigarh<br>Confirmed: 59577<br>Deaths: 735<...
6,Chhattisgarh,21.264705,82.035366,965208,12915,905361,2021-05-29 07:20:52,Chhattisgarh<br>Confirmed: 965208<br>Deaths: 1...
7,Dadra and Nagar Haveli,20.194742,73.080901,10207,4,9845,2021-05-29 07:20:52,Dadra and Nagar Haveli<br>Confirmed: 10207<br>...
8,Delhi,28.646519,77.108980,1423690,23951,1385158,2021-05-29 07:20:52,Delhi<br>Confirmed: 1423690<br>Deaths: 23951<b...
9,Goa,15.359682,74.057396,153456,2570,135560,2021-05-29 07:20:52,Goa<br>Confirmed: 153456<br>Deaths: 2570<br>Re...


In [12]:
# TOTAL CASES WORLDWIDE
df_total_worldwide = df_final.groupby("Country_Region", as_index=False).agg(
    {
        "Confirmed" : "sum",
        "Deaths" : "sum",
        "Recovered" : "sum"
    }
)

df_total_worldwide

,Country_Region,Confirmed,Deaths,Recovered
0,Afghanistan,69130,2881,57119
1,Albania,132285,2448,129097
2,Algeria,128198,3455,89232
3,Andorra,13693,127,13416
4,Angola,33944,749,27577
...,...,...,...,...
188,Vietnam,6713,47,2896
189,West Bank and Gaza,307569,3489,300125
190,Yemen,6723,1316,3375
191,Zambia,94430,1275,91443


In [13]:
df_total_india = df_total_worldwide[df_total_worldwide['Country_Region'] == 'India']
df_total_india

,Country_Region,Confirmed,Deaths,Recovered
79,India,27729247,322512,25178011


In [14]:
total_confirmed = df_india["Confirmed"].sum()
total_recovered = df_india["Recovered"].sum()
total_deaths = df_india["Deaths"].sum()

In [15]:
fig = make_subplots(
    rows = 4, cols = 6,
    specs=[
            [{"type": "scattergeo", "rowspan": 4, "colspan": 3}, None, None, {"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"} ],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,              {"type": "bar", "colspan":3}, None, None],
            [    None, None, None,               {"type": "bar", "colspan":3}, None, None],
          ]
)

In [16]:
message = df_india["State"] + "<br>"
message += "Confirmed: " + df_india["Confirmed"].astype(str) + "<br>"
message += "Deaths: " + df_india["Deaths"].astype(str) + "<br>"
message += "Recovered: " + df_india["Recovered"].astype(str) + "<br>"
message += "Last updated: " + df_india["Last_Update"].astype(str)
df_india["text"] = message

In [17]:

india_states = pd.read_json('https://raw.githubusercontent.com/nikhilkumarsingh/choropleth-python-tutorial/master/states_india.geojson')
india_states

,type,features
0,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
1,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
2,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
3,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
4,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
5,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
6,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
7,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
8,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."
9,FeatureCollection,"{'type': 'Feature', 'geometry': {'type': 'Mult..."


In [18]:
#MAP THE DATAFRAMES FOR MAP PLOTTING

state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]

state_id_map

{'Andaman & Nicobar Island': 35,
 'Andhra Pradesh': 28,
 'Arunanchal Pradesh': 12,
 'Assam': 18,
 'Bihar': 10,
 'Chandigarh': 4,
 'Chhattisgarh': 22,
 'Dadara & Nagar Havelli': 26,
 'Daman & Diu': 25,
 'Goa': 30,
 'Gujarat': 24,
 'Haryana': 6,
 'Himachal Pradesh': 2,
 'Jammu & Kashmir': 1,
 'Jharkhand': 20,
 'Karnataka': 29,
 'Kerala': 32,
 'Lakshadweep': 31,
 'Madhya Pradesh': 23,
 'Maharashtra': 27,
 'Manipur': 14,
 'Meghalaya': 17,
 'Mizoram': 15,
 'NCT of Delhi': 7,
 'Nagaland': 13,
 'Odisha': 21,
 'Puducherry': 34,
 'Punjab': 3,
 'Rajasthan': 8,
 'Sikkim': 11,
 'Tamil Nadu': 33,
 'Telangana': 0,
 'Tripura': 16,
 'Uttar Pradesh': 9,
 'Uttarakhand': 5,
 'West Bengal': 19}

In [21]:

state_id_map['Andaman and Nicobar'] = state_id_map['Andaman & Nicobar Island']
del state_id_map['Andaman & Nicobar Island']

In [23]:
state_id_map['Arunachal Pradesh'] = state_id_map['Arunanchal Pradesh']
del state_id_map['Arunanchal Pradesh']

In [27]:
state_id_map['Dadra and Nagar Haveli'] = state_id_map['Dadara & Nagar Havelli']
del state_id_map['Dadara & Nagar Havelli']

In [29]:
state_id_map['Delhi'] = state_id_map['NCT of Delhi']
del state_id_map['NCT of Delhi']

In [31]:
state_id_map['Jammu and Kashmir'] = state_id_map['Jammu & Kashmir']
del state_id_map['Jammu & Kashmir']

In [38]:
df_india['id'] = df_india['State'].apply(lambda x : state_id_map[x])

CHOROPLETH MAP

In [39]:
fig = px.choropleth(
    df_india,
    locations="id",
    geojson=india_states,
    color="Confirmed",
    hover_name="State",
    hover_data=["Confirmed"],
    title="India COVID-19 Cases",
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

TypeError: ignored

In [ ]:
fig.add_trace(
    go.Indicator(
        mode="number",
        value= total_confirmed,
        title="Confirmed",
    ),
    row=1, col=4
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_recovered,
        title="Recovered",
    ),
    row=1, col=5
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=total_deaths,
        title="Deaths",
    ),
    row=1, col=6
)